In [2]:
import pandas as pd
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn import metrics

In [3]:
North_east_2 = pd.read_csv("North_east_2.csv")
North_east = pd.read_csv("North_east.csv")
North_west_2 = pd.read_csv("North_west_2.csv")
North_west_1 = pd.read_csv("North_west_1.csv")

In [4]:

dfs = [North_east_2, North_east,North_west_2,North_west_1]
df=pd.DataFrame()


for i in dfs:
    df = df.append(i)

data_mean=df.groupby('Datetime').mean()
data_mean.index = pd.to_datetime(data_mean.index)

In [5]:
data_mean.columns

Index(['air_temp_5cm_avg', 'air_temp_5cm_max', 'air_temp_5cm_min',
       'air_temp_avg', 'air_temp_max', 'air_temp_min', 'cloud_cover_avg',
       'cloud_cover_max', 'cloud_cover_min', 'dew_point_avg', 'dew_point_max',
       'dew_point_min', 'ice_acc_period', 'liquid_acc_period',
       'long_wave_radiation_avg', 'long_wave_radiation_max',
       'long_wave_radiation_min', 'pet_period', 'precip_acc_period',
       'precip_acc_period_adjusted', 'precip_acc_period_raw',
       'precip_duration', 'relative_humidity_avg', 'relative_humidity_max',
       'relative_humidity_min', 'short_wave_radiation_avg',
       'short_wave_radiation_max', 'short_wave_radiation_min',
       'snow_acc_period', 'sunshine_duration', 'visibility_avg',
       'visibility_max', 'visibility_min', 'wind_gust_max',
       'wind_speed_2m_avg', 'wind_speed_2m_max', 'wind_speed_2m_min',
       'wind_speed_avg', 'wind_speed_max', 'wind_speed_min'],
      dtype='object')

In [6]:
price =  pd.read_csv("London Wheat Futures Historical Data.csv")
price['Datetime'] = pd.to_datetime(price['Date'])
price = price.set_index(['Datetime'])
price = price.drop(['Date'], axis=1)
price =price.groupby(pd.Grouper(freq='10D'))['Price'].mean()

In [8]:

def rf_r2score(var,price):
    features = pd.get_dummies(var)
    labels = price
    features = np.array(features)
    labels = np.array(labels)

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=100)

    rf_exp = RandomForestRegressor(n_estimators= 200, random_state=1000)
    rf_exp.fit(X_train, y_train)

    y_pred = rf_exp.predict(X_test)
    print("R2: ",r2_score(y_test,y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [40]:
data_copy = data_mean

data_copy['frost_days'] = data_mean[data_mean.air_temp_min<0 ]['air_temp_min']
data_copy['ice_days'] = data_mean[data_mean.air_temp_max<0 ]['air_temp_max']
data_copy['tem_10_25'] = data_mean[(10<data_mean.air_temp_avg)&(data_mean.air_temp_avg<25) ]['air_temp_avg']
data_copy['wet_days'] = data_mean[data_mean.precip_acc_period>1 ]['precip_acc_period']
data_copy['very_heavy_precip_days'] = data_mean[data_mean.precip_acc_period>20 ]['precip_acc_period']
data_copy['heavy_precip_days'] = data_mean[data_mean.precip_acc_period>10 ]['precip_acc_period']
data_copy['Summer_days'] = data_mean[data_mean.air_temp_max>25 ]['air_temp_max']
data_copy['precip_sum'] = data_mean['precip_acc_period']
data_copy['tropical_nights'] = data_mean[data_mean.air_temp_min>20 ]['air_temp_min']
data_1 = data_copy.groupby(pd.Grouper(freq='10D')).agg({'frost_days':'count', 
                                               'ice_days':'count',
                                               'tem_10_25':'count',
                                               'very_heavy_precip_days':'count',
                                               'heavy_precip_days':'count',
                                               'Summer_days':'count',
                                               'wet_days':'count',
                                                'cloud_cover_avg':'sum',
                                               'precip_sum':'sum', 
                                              'sunshine_duration':'sum',
                                              'tropical_nights':'count',
                                              'air_temp_max':['max','min','mean'],'air_temp_min':['max','min','mean'],
                                              'air_temp_avg':'mean'
                         })

winter =data_1.loc[(data_1.index.month.isin([11, 12, 1, 2, 3]))]
winter_price = price.loc[(price.index.month.isin([11, 12, 1, 2, 3]))]
print(winter.corrwith(winter_price))
print(rf_r2score(winter_price,winter))

frost_days              count    0.081265
ice_days                count    0.122242
tem_10_25               count   -0.086261
very_heavy_precip_days  count   -0.128711
heavy_precip_days       count   -0.014160
Summer_days             count         NaN
wet_days                count   -0.121729
cloud_cover_avg         sum      0.332685
precip_sum              sum     -0.090219
sunshine_duration       sum      0.059277
tropical_nights         count         NaN
air_temp_max            max     -0.015702
                        min     -0.151064
                        mean    -0.103692
air_temp_min            max     -0.003055
                        min     -0.063806
                        mean    -0.098962
air_temp_avg            mean    -0.098737
dtype: float64
R2:  0.09694227755792761
Root Mean Squared Error: 51.30789095049996
None


In [41]:
data_1 = data_copy.groupby(pd.Grouper(freq='10D')).agg({'frost_days':'count', 
                                               'ice_days':'count',
                                               'tem_10_25':'count',
                                               'very_heavy_precip_days':'count',
                                               'heavy_precip_days':'count',
                                               'Summer_days':'count',
                                               'wet_days':'count',
                                               'precip_sum':'sum', 
                                              'sunshine_duration':'sum',
                                              'tropical_nights':'count',
                                              'air_temp_max':['max','min','mean'],'air_temp_min':['max','min','mean'],
                                              'air_temp_avg':'mean'
                         })
winter =data_1.loc[(data_1.index.month.isin([11, 12, 1, 2, 3]))]
print(winter.corrwith(winter_price))
print(rf_r2score(winter_price,winter))

frost_days              count    0.081265
ice_days                count    0.122242
tem_10_25               count   -0.086261
very_heavy_precip_days  count   -0.128711
heavy_precip_days       count   -0.014160
Summer_days             count         NaN
wet_days                count   -0.121729
precip_sum              sum     -0.090219
sunshine_duration       sum      0.059277
tropical_nights         count         NaN
air_temp_max            max     -0.015702
                        min     -0.151064
                        mean    -0.103692
air_temp_min            max     -0.003055
                        min     -0.063806
                        mean    -0.098962
air_temp_avg            mean    -0.098737
dtype: float64
R2:  0.11962270004154757
Root Mean Squared Error: 5.167309674100163
None


In [42]:
data_1 = data_copy.groupby(pd.Grouper(freq='10D')).agg({'frost_days':'count', 
                                               'ice_days':'count',
                                               'tem_10_25':'count',
                                               'very_heavy_precip_days':'count',
                                               'heavy_precip_days':'count',
                                                'cloud_cover_avg':'sum',
                                               'wet_days':'count',
                                               'precip_sum':'sum', 
                                              'sunshine_duration':'sum',
                                              
                                              'air_temp_max':['max','min','mean'],'air_temp_min':['max','min','mean'],
                                              'air_temp_avg':'mean'
                         })
winter =data_1.loc[(data_1.index.month.isin([11, 12, 1, 2, 3]))]
print(winter.corrwith(winter_price))
print(rf_r2score(winter_price,winter))

frost_days              count    0.081265
ice_days                count    0.122242
tem_10_25               count   -0.086261
very_heavy_precip_days  count   -0.128711
heavy_precip_days       count   -0.014160
cloud_cover_avg         sum      0.332685
wet_days                count   -0.121729
precip_sum              sum     -0.090219
sunshine_duration       sum      0.059277
air_temp_max            max     -0.015702
                        min     -0.151064
                        mean    -0.103692
air_temp_min            max     -0.003055
                        min     -0.063806
                        mean    -0.098962
air_temp_avg            mean    -0.098737
dtype: float64
R2:  -0.015939937747331434
Root Mean Squared Error: 54.420236429217624
None


In [25]:
corr = winter.corr()
corr.style.background_gradient(cmap='coolwarm')

/Users/yihangu/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/style.py:1126: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(s.to_numpy()) if vmin is None else vmin
/Users/yihangu/opt/anaconda3/lib/python3.8/site-packages/pandas/io/formats/style.py:1127: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(s.to_numpy()) if vmax is None else vmax


In [34]:
data_spring = data_copy.groupby(pd.Grouper(freq='10D')).agg({'frost_days':'count', 
                                               
                                               'tem_10_25':'count',
                                               'very_heavy_precip_days':'count',
                                               'heavy_precip_days':'count',
                                               'Summer_days':'count',
                                               'wet_days':'count',
                                               'precip_sum':'sum', 
                                              'sunshine_duration':'sum',
                                              'tropical_nights':'count',
                                              'air_temp_max':['max','min','mean'],'air_temp_min':['max','min','mean'],
                                              'air_temp_avg':'mean'
                         })

spring =data_spring.loc[(data_spring.index.month.isin([4,5]))]
spring_price = price.loc[(price.index.month.isin([4,5]))]
print(spring.corrwith(spring_price))
print(rf_r2score(spring_price,spring))



frost_days              count    0.143518
tem_10_25               count    0.126257
very_heavy_precip_days  count   -0.006345
heavy_precip_days       count    0.076416
Summer_days             count   -0.057291
wet_days                count   -0.028974
precip_sum              sum      0.009096
sunshine_duration       sum      0.418697
tropical_nights         count         NaN
air_temp_max            max      0.031794
                        min      0.155579
                        mean     0.093092
air_temp_min            max      0.101512
                        min     -0.001411
                        mean     0.075769
air_temp_avg            mean     0.077643
dtype: float64
R2:  0.050174160085354594
Root Mean Squared Error: 8.241803135209542
None


In [35]:
data_summer = data_copy.groupby(pd.Grouper(freq='10D')).agg({ 
                                               'frost_days':'count', 
                                               'ice_days':'count',
                                               'tem_10_25':'count',
                                               'very_heavy_precip_days':'count',
                                               'heavy_precip_days':'count',
                                               'Summer_days':'count',
                                               'wet_days':'count',
                                               'precip_sum':'sum', 
                                              'sunshine_duration':'sum',
                                              'tropical_nights':'count',
                                              'air_temp_max':['max','min','mean'],'air_temp_min':['max','min','mean'],
                                              'air_temp_avg':'mean'
                         })



summer =data_summer.loc[(data_summer.index.month.isin([6,7]))]
summer_price = price.loc[(price.index.month.isin([6,7]))]
print(summer.corrwith(summer_price))
print(rf_r2score(summer_price,summer))




frost_days              count         NaN
ice_days                count         NaN
tem_10_25               count    0.192681
very_heavy_precip_days  count    0.002853
heavy_precip_days       count    0.126702
Summer_days             count    0.110207
wet_days                count    0.202822
precip_sum              sum      0.213097
sunshine_duration       sum      0.346832
tropical_nights         count         NaN
air_temp_max            max     -0.001895
                        min     -0.033332
                        mean    -0.047414
air_temp_min            max     -0.113711
                        min     -0.059166
                        mean    -0.028206
air_temp_avg            mean    -0.054537
dtype: float64
R2:  0.09163253241682873
Root Mean Squared Error: 7.525476441227654
None
